In [1]:
%cd ..

/home/ltorres/leo/tesis/cloud-classification/code


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from torchvision.io import read_image
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from random import sample
import torch
import torch.nn as nn

In [4]:
from src.dataset import GCD
from src import config
from src import engine_gnn as engine
from src import utils

from src.models.graph_nets import GraphConvGNN

In [5]:
from sklearn.metrics import accuracy_score

In [6]:
device = config.DEVICE

#### Data loaders

In [7]:
path_train_images = utils.get_gcd_paths(config.DATA_DIR,'train')

In [8]:
train_dataset = GCD(path_train_images, resize=256)

train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=4,
        shuffle=True,
    )

In [9]:
path_test_images = utils.get_gcd_paths(config.DATA_DIR,'test')

test_dataset = GCD(path_test_images, resize=256)

test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE,
        num_workers=4,
        shuffle=False,
    )

### Data augmentation data loaders

In [10]:
augmentation_datasets = [
    GCD(utils.random_sample(path_train_images), resize=256, aug_types= atype)
    for atype in config.AUGMENTATION_TYPES
    ]

In [11]:
augmentation_loaders = [
    torch.utils.data.DataLoader(
            aug_dataset,
            batch_size=config.BATCH_SIZE,
            num_workers=4,
            shuffle=True,
        )
for aug_dataset in augmentation_datasets
]

#### Model

In [12]:
from torchvision import models

In [13]:
torch.cuda.empty_cache()

In [14]:
model = GraphConvGNN(7).to(config.DEVICE)

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
best_accuracy=0
best_iteration=0
best_params={}

for e in range(config.EPOCHS):
    ### TRAIN DATASET
    preds, targets, loss = engine.train_fn(model, train_loader, criterion, optimizer, device=device)
    train_acc = accuracy_score(targets, preds)
    
    ### AUGMENTATION IMAGES
    for _loader in augmentation_loaders:
        engine.forward_backward_pass(model, _loader, criterion, optimizer, device=device)
    
    test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device=device)
    test_acc = accuracy_score(test_targets, test_preds)
    
    if test_acc > best_accuracy:
        print(f"Saved best parameters at epoch {e+1}")
        best_accuracy = test_acc
        best_iteration = e+1
        best_params = model.state_dict()
    
    print("EPOCH {}: Train acc: {:.2%} Train Loss: {:.4f} Test acc: {:.2%} Test Loss: {:.4f}".format(
        e+1,
        train_acc,
        loss,
        test_acc,
        test_loss
    ))


100%|██████████| 282/282 [00:17<00:00, 16.01it/s]


Saved best parameters at epoch 1
EPOCH 1: Train acc: 79.53% Train Loss: 1.1304 Test acc: 73.40% Test Loss: 1.6568


100%|██████████| 282/282 [00:17<00:00, 16.03it/s]


EPOCH 2: Train acc: 88.21% Train Loss: 0.6299 Test acc: 71.04% Test Loss: 2.0220


100%|██████████| 282/282 [00:17<00:00, 15.99it/s]


Saved best parameters at epoch 3
EPOCH 3: Train acc: 89.38% Train Loss: 0.5443 Test acc: 74.24% Test Loss: 1.8765


100%|██████████| 282/282 [00:17<00:00, 16.01it/s]


Saved best parameters at epoch 4
EPOCH 4: Train acc: 91.48% Train Loss: 0.4407 Test acc: 75.33% Test Loss: 1.7201


 22%|██▏       | 35/157 [00:06<00:21,  5.73it/s]

In [ ]:
torch.save(best_params, config.SAVE_PATH+'/gcd_gnn/AUGMENTATION_model_params_25_03_22.pt')

In [ ]:
model.load_state_dict(best_params)

In [ ]:
test_preds, test_targets, test_loss = engine.eval_fn(model, test_loader, criterion, device=device)

In [ ]:
accuracy_score(test_targets, test_preds)

---

### Accuracy per class and confusion matrix

In [ ]:
class_mapper = {
    0: '1_cumulus',
    1: '2_altocumulus',
    2: '3_cirrus',
    3: '4_clearsky',
    4: '5_stratocumulus',
    5: '6_cumulonimbus',
    6: '7_mixed',
}

In [ ]:
class_acc = dict()
matrix = np.zeros((7,7), dtype=int)

for i in range(7):
    pred_index = np.where(test_targets==i)[0]
    class_acc[class_mapper[i]] = accuracy_score(test_targets[pred_index], test_preds[pred_index])
    
    for j in test_preds[pred_index]:
        matrix[i,j]+=1 

In [ ]:
class_acc

In [ ]:
sum_of_rows = matrix.sum(axis=1)
conf_mat = 100*matrix / sum_of_rows[:, np.newaxis]

df_cm = pd.DataFrame(conf_mat, index = class_acc.keys(),
                  columns = class_acc.keys())

plt.figure(figsize = (8,5))
plt.title("Confusion Matrix (Accuracy %)")
sns.heatmap(df_cm, annot=True)